In [ ]:
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re
from tqdm import tqdm
from peft import PeftModel
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/gemma_math_project/10623Project

In [1]:

# Load the model and tokenizer
#smodel_name = "google/gemma-7b"  # Instruction-tuned Gemmas


# 2) point to your model root and adapter folder
#base_model_path = "/content/drive/MyDrive/models/qlora_gemma_gsm8k_reasoning/checkpoint-699"
adapter_path    = "/content/drive/MyDrive/gemma_math_project/10623Project/qlora_gemma_gsm8k/adapter"


# 3) load tokenizer + base model locally
model_name = "google/gemma-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
base_model = AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype=torch.bfloat16,
                                             device_map="cuda:0",
                                             token=hf_token)

# 4) overlay your locally‑saved LoRA adapter
model = PeftModel.from_pretrained(
    base_model,
    adapter_path,
    is_trainable=False,       # inference only
    local_files_only=True     # only look locally for the adapter
)

model.eval()

NameError: name 'AutoTokenizer' is not defined

In [ ]:
# Load test data
with open("gsm8k_reasoning_test.jsonl", "r") as f:  # change to your actual file path
    test_data = [json.loads(line) for line in f]

# Step 1: Extract the first answer block (robust to repeated Q/A)
def extract_first_answer_block(text):
    parts = text.split("A:", 1)
    if len(parts) < 2:
        return text.strip()
    answer_section = parts[1].strip()
    stop_index = answer_section.find("Q:")
    if stop_index != -1:
        answer_section = answer_section[:stop_index].strip()
    return answer_section

# Step 2: Extract final answer
def extract_final_answer(text):
    match = re.search(r"####\s*([-\d.,]+)", text)
    if match:
        return match.group(1).replace(",", "")
    numbers = re.findall(r"[-+]?\d*\.?\d+", text)
    return numbers[-1] if numbers else None

# Step 3: Normalize answers
def normalize_answer(ans):
    if ans is None:
        return None
    ans = ans.replace(",", "").replace("$", "").strip()
    try:
        return str(int(float(ans)))
    except:
        return ans

# Inference in batches
batch_size = 16
results = []

for i in tqdm(range(0, len(test_data), batch_size)):
# for i in tqdm(range(0, 4, batch_size)):
    batch = test_data[i:i+batch_size]
    # prompts = []
    # for item in batch:
    #     question = item["prompt"].replace("Q:", "").replace("A:", "").strip()
    #     new_prompt = f"Q: {question}\n\nWrite the final answer like this: #### [answer].\n\nA:"
    #     prompts.append(new_prompt)

    prompts = [item["prompt"] for item in batch]
    # print("Prompts: ", prompts)
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=False,
            temperature=0.0,
            pad_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    for j, output in enumerate(decoded):
        prediction = extract_first_answer_block(output)
        pred_ans = extract_final_answer(prediction)
        true_ans = extract_final_answer(batch[j]["completion"])
        correct = normalize_answer(pred_ans) == normalize_answer(true_ans)
        # print(f"Question: {prompts[j]}")
        # print(f"Output: {output}")
        # print(f"Prediction: {prediction}")
        # print(f"True Answer: {true_ans}")
        # print(f"Predicted Answer: {pred_ans}")

        results.append({
            "question": batch[j]["prompt"],
            "prediction": prediction,
            "pred_ans": pred_ans,
            "true_ans": true_ans,
            "correct": correct
        })

# Accuracy report
accuracy = sum(r["correct"] for r in results) / len(results)
print(f"\n✅ Baseline Accuracy: {accuracy:.2%}")